In [3]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt

#import regex and math library
import re
import math



In [4]:
df = pd.read_csv('../data/Annual_Temperature.csv')

def getCountryModelTemp(country_code):
    #regex expression used to get all columns containing yearly data
    # column headers include F2000, F2001, F2002, F1987 so on
    #\d indicates integer in the regular expression
    regex_expr = "F\d\d\d\d"
    
    #storing all column names of the data frame in a list
    col_names = []
    for col in df.columns:
        col_names.append(col)

    #storing the column names in a string which will be used for the regular expression    
    string = ""
    for col in col_names:
        string += col

    #using the regular expression to get a list containing only the yearly values headers (e.g. F2000, F2001)   
    listReg = re.findall(regex_expr, string)
    
    # storing only the yearly values, object id, and Country headers in a new list
    dropped_cols = []
    for item in col_names: 
        if listReg.count(item) == 0 and item != 'ObjectId' and item != 'Country':
            dropped_cols.append(item)

    # dropping unused columns
    modified_df = df
    for item in dropped_cols:
        modified_df = modified_df.drop(item, axis = 1)


    # new, separate data frame without the object id and country headers
    row = modified_df[modified_df['ObjectId'] == country_code]
    row = row.drop('ObjectId', axis = 1)
    row = row.drop('Country', axis = 1)


    # getting the values in a 1D array
    temp_changes = row.values.tolist()
    temp_changes = temp_changes[0]


    #1961 is the start year, and thus our start index
    start = 1961
    x_values = []
    y_values = []

    for item in temp_changes:
        x_values.append(start)
        # append if not nan/null
        if not math.isnan(item):
            y_values.append(item)
        start += 1

    # getting the y = mx + b model 
    outer = []
    for i in range(len(y_values)):
        for x, y in zip(x_values, y_values):
            outer.append([x, y])

    dt = np.array(outer)
    x = dt[:, 0]
    y = dt[:, 1]

    theta = np.polyfit(x, y, 1)
    name = df['Country'].values[country_code - 1]
    model_value = f'The model of {name}: {theta}'


    return model_value

In [5]:
print(getCountryModelTemp(117))

The model of Lithuania: [-1.53523810e-02  3.75312858e+01]
